In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import ipywidgets as widgets

In [17]:
df = pd.read_csv('results/multi_gpt.csv')
df

,Unnamed: 0,P,Num,Origin,gpt_A_normal,gpt_B_normal,gpt_A_flipped,gpt_B_flipped,gpt_A_count,gpt_B_count,gpt_choice,gpt_flipped_choice,Response_A,Response_B
0,0,"Evan, an American tourist with a deep apprecia...",0,A,1.0,0.0,1.0,0.0,1.0,0.0,A,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
1,1,"Evan, an American tourist with a keen eye for ...",0,B,0.0,1.0,0.0,1.0,0.0,1.0,B,B,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
2,2,"Evan, an American traveler with a profound adm...",1,A,1.0,0.0,1.0,0.0,1.0,0.0,A,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
3,3,"Evan, an American traveler known for his disce...",1,B,0.0,1.0,0.0,1.0,0.0,1.0,B,B,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
4,4,"Evan, an American traveler with a deep appreci...",2,A,1.0,0.0,1.0,0.0,1.0,0.0,A,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,"At Paris’ annual, city-wide spring festival – ...",47,B,0.0,1.0,1.0,0.0,0.5,0.5,B,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
96,96,"While savoring his time in France, particularl...",48,A,0.0,1.0,1.0,0.0,0.5,0.5,B,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
97,97,"At Paris' annual, city-wide spring festival - ...",48,B,0.0,1.0,1.0,0.0,0.5,0.5,B,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
98,98,"While savoring his time in France, particularl...",49,A,0.0,1.0,1.0,0.0,0.5,0.5,B,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."


In [18]:
def summary(prefix=""):
    df[f'{prefix}correct'] = df['Origin'] == df[f'{prefix}choice']
    df[f'{prefix}flipped_correct'] = df['Origin'] == df[f'{prefix}flipped_choice']
    df[f'{prefix}avg_correct'] = (df[f'{prefix}correct'].astype(int)+df[f'{prefix}flipped_correct'].astype(int))/2
    print(f"Correct: {df[f'{prefix}correct'].mean()}, Flipped Correct: {df[f'{prefix}flipped_correct'].mean()}, Avg Correct: {df[f'{prefix}avg_correct'].mean()}")

summary("gpt_")

Correct: 0.73, Flipped Correct: 0.73, Avg Correct: 0.73


In [20]:
dropdown = widgets.Dropdown(
    options=['avg_correct', 'correct', 'flipped_correct'],
    value='avg_correct',
    description='Statistic:',
)

origin = widgets.Dropdown(
    options=['A', 'B', 'Both'],
    value='Both',
    description='Origin:',
)

slider = widgets.IntSlider(min=1, max=20, step=1, value=1)

def plot(prefix=""):
    match origin.value:
        case 'A':
            data = df[df['Origin'] == 'A'][f'{prefix}{dropdown.value}'].reset_index(drop=True)
            # fall through
        case 'B':
            data = df[df['Origin'] == 'B'][f'{prefix}{dropdown.value}'].reset_index(drop=True)
        case 'Both':
            tmp1 = df[df['Origin'] == 'A'][f'{prefix}{dropdown.value}'].reset_index(drop=True)
            tmp2 = df[df['Origin'] == 'B'][f'{prefix}{dropdown.value}'].reset_index(drop=True)
            data = (tmp1+tmp2)/2
    
    rolling_mean = data.rolling(window=slider.value).mean()
    plt.title(f'{prefix}{dropdown.value} (window={slider.value}, origin={origin.value})')
    plt.xlabel('Mutation number')
    plt.ylabel('Accuracy')
    plt.plot(rolling_mean, color='orange')
    plt.show()

widgets.interact(lambda x, y, z: plot("gpt_"), x=dropdown, y=slider, z=origin)

interactive(children=(Dropdown(description='Statistic:', options=('avg_correct', 'correct', 'flipped_correct')…

<function __main__.<lambda>(x, y, z)>